In [ ]:
!pip install tensorflow keras opencv-python matplotlib

import os, zipfile, shutil, random
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models


In [ ]:

from google.colab import files
uploaded = files.upload()


Saving archive.zip to archive.zip


In [ ]:

zip_path = list(uploaded.keys())[0]

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall("dataset")

print("Extracted Successfully!")


Extracted Successfully!


In [ ]:
import os, shutil

TARGET_CLASSES = [
    'Potato___Early_blight',
    'Potato___Late_blight',
    'Potato___healthy'
]


if len(os.listdir("dataset/PlantVillage")) > 5:
    SOURCE_DIR = "dataset/PlantVillage"
elif len(os.listdir("dataset/plantvillage")) > 5:
    SOURCE_DIR = "dataset/plantvillage"
else:
    raise Exception("Dataset folder does not contain images.")

print("Using SOURCE_DIR =", SOURCE_DIR)

TARGET_DIR = "Filtered"
os.makedirs(TARGET_DIR, exist_ok=True)


for cls in TARGET_CLASSES:
    src = os.path.join(SOURCE_DIR, cls)

    if not os.path.exists(src):
        print(f"WARNING: Class folder not found: {cls}")
        continue

    dst = os.path.join(TARGET_DIR, cls)
    os.makedirs(dst, exist_ok=True)

    for img in os.listdir(src):
        shutil.copy(os.path.join(src, img), os.path.join(dst, img))

print("Filtering completed!")


Using SOURCE_DIR = dataset/PlantVillage
Filtering completed!


In [ ]:

def make_split(src_root, dst_root="Split"):
    if os.path.exists(dst_root):
        shutil.rmtree(dst_root)

    for split in ["train", "val", "test"]:
        for cls in TARGET_CLASSES:
            os.makedirs(os.path.join(dst_root, split, cls), exist_ok=True)

    for cls in TARGET_CLASSES:
        files = os.listdir(os.path.join(src_root, cls))
        random.shuffle(files)

        train, val, test = np.split(
            files,
            [int(0.7*len(files)), int(0.85*len(files))]
        )

        for f in train:
            shutil.copy(os.path.join(src_root, cls, f), f"{dst_root}/train/{cls}/{f}")

        for f in val:
            shutil.copy(os.path.join(src_root, cls, f), f"{dst_root}/val/{cls}/{f}")

        for f in test:
            shutil.copy(os.path.join(src_root, cls, f), f"{dst_root}/test/{cls}/{f}")

    print("Dataset split created successfully!")

make_split(TARGET_DIR)


Dataset split created successfully!


In [ ]:

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train = train_gen.flow_from_directory(
    "Split/train",
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

val = val_gen.flow_from_directory(
    "Split/val",
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

test = test_gen.flow_from_directory(
    "Split/test",
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)


Found 1506 images belonging to 3 classes.
Found 323 images belonging to 3 classes.
Found 323 images belonging to 3 classes.


In [ ]:

base = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base.trainable = False  # freeze for fast training

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.35),
    layers.Dense(len(TARGET_CLASSES), activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │         3,843 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,053,414 (15.46 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:

history = model.fit(
    train,
    validation_data=val,
    epochs=10,
    verbose=1
)


Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 148s 3s/step - accuracy: 0.4399 - loss: 0.9363 - val_accuracy: 0.4644 - val_loss: 0.8999
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 146s 3s/step - accuracy: 0.4615 - loss: 0.8848 - val_accuracy: 0.4644 - val_loss: 0.9190
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.4639 - loss: 0.9252 - val_accuracy: 0.4644 - val_loss: 0.9001
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 159s 3s/step - accuracy: 0.4479 - loss: 0.8989 - val_accuracy: 0.4644 - val_loss: 0.9007
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 149s 3s/step - accuracy: 0.4992 - loss: 0.9039 - val_accuracy: 0.4644 - val_loss: 0.9010
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 148s 3s/step - accuracy: 0.4374 - loss: 0.9277 - val_accuracy: 0.4644 - val_loss: 0.9021
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 149s 3s/step - accuracy: 0.4629 - loss: 0.9143 - val_accuracy: 0.4644 - val_loss: 0.9179
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.4674 - loss: 0.9257 - val_accuracy: 0.4644 - v

In [ ]:

loss, acc = model.evaluate(test)
print("TEST ACCURACY =", acc)


11/11 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.4281 - loss: 0.9113
TEST ACCURACY = 0.46439629793167114
